In [ ]:
import json
import csv
import os
import torch
import sys
import tqdm

sys.path.append('..')
from tools.TextProcessing import build_word_tree, process_keywords
from tools.BasicUtils import my_write, my_csv_read, my_read, my_json_read, ntopidx, batch
from joint_score_func import *

In [ ]:
file_description = [
    "keyword_f.txt ---- CS keywords",
    "wordtree.json ---- word tree for cs keywords",
    "entity.txt ---- Reformed cs keywords with '_' replacing ' '"
]
    
my_write('readme.txt', file_description)

## Load data

In [ ]:
eid2ent_dict = my_json_read('data/eid2ent.json')
rid2rel_dict = my_json_read('data/rid2rel.json')
kg_cs_triples = my_csv_read('data/kg_cs_triples.csv')
rel_list = list(set(rid2rel_dict.values()))

## Setup model

In [ ]:
sparse_retriever = SparseRetrieveSentForPairCoOccur('../data/corpus/small_sent.txt', 'data/occur.json')

In [ ]:
test_sent = sparse_retriever.retrieve('java', 'python')
print(len(test_sent))
print(test_sent[0])

In [ ]:
sf1 = ScoreFunction1()

In [ ]:
sf2 = ScoreFunction2()

In [ ]:
# ScoreFunction Example
scores, candidate_sents = demo_score_function('python', 'java', sf2, sparse_retriever)
print(scores)
print('\n'.join(candidate_sents))

In [ ]:
dense_retriever = DenseRetrieverWithScore(sparse_retriever, sf2)

In [ ]:
sents, scores = dense_retriever(['python', 'data'], ['java', 'database'])
print(sents.shape)
print(scores.shape)

In [ ]:
reader1 = Reader1(rel_list)

In [ ]:
# Reader1 Example
rels = demo_reader('python', 'java', reader1, sparse_retriever)
print(rels)

In [ ]:
reader3 = Reader3()

## Training

In [ ]:
sents, scores = dense_retriever(['python', 'data'], ['java', 'database'])

In [ ]:
with my_tokenizer.as_target_tokenizer():
    target = my_encode(['is like', 'is in'])

In [ ]:
output, loss = reader3(sents['input_ids'], scores, target['input_ids'])

In [ ]:
loss.backward()

In [ ]:
train()